# Phi-4 Mini ONNX പാരലൽ ഫംഗ്ഷൻ കോളിംഗ് ട്യൂട്ടോറിയൽ

This notebook demonstrates how to use Phi-4 Mini with ONNX Runtime GenAI for parallel function calling. Function calling allows the model to intelligently invoke external tools and APIs based on user requests.

## അവലോകനം

In this tutorial, you'll learn how to:
- Phi-4 Mini ONNX Runtime GenAI-നൊപ്പം സജ്ജമാക്കുക
- ഫ്ലൈറ്റുകളും ഹോട്ടലുകളും ബുക്ക് ചെയ്യുന്നതിനായുള്ള ഫംഗ്ഷൻ സ്കീമുകൾ നിർവചിക്കുക
- സംരചിത ഫലങ്ങൾക്കായി Lark വ്യാകരണത്തോടുകൂടി മാർഗ്ഗനിർദേശമുള്ള ജനറേഷൻ ഉപയോഗിക്കുക
- സങ്കീർണ യാത്രാ ബുക്കിംഗ് സാഹചര്യങ്ങൾക്കായി പാരലൽ ഫംഗ്ഷൻ കോൾകൾ നടപ്പിലാക്കുക

## ആവശ്യകതകൾ

Before running this notebook, ensure you have:
- Phi-4 Mini ONNX മോഡൽ ഡൗൺലോഡ് ചെയ്തിട്ടുണ്ട്
- Installed `onnxruntime-genai` package
- ഫംഗ്ഷൻ കോളിംഗ് ആശയങ്ങളുടെ അടിസ്ഥാനപരമായ ബോധം


## പടി 1: ആവശ്യമായ ലൈബ്രറികൾ ഇറക്കുമതി ചെയ്യുക

ആദ്യം, ഫങ്ഷൻ കോൾ ചെയ്യൽ നടപ്പാക്കുന്നതിനായി ആവശ്യമായ ലൈബ്രറികൾ നമുക്ക് ഇറക്കുമതി ചെയ്യാം.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## പടി 2: മോഡൽ സജ്ജീകരണവും കോൺഫിഗറേഷനും

ഇപ്പോൾ നാം Phi-4 Mini ONNX മോഡൽ ക്രമീകരിക്കും. പാത നിങ്ങളുടെ യഥാർത്ഥ മോഡൽ ഡയറക്ടറിയായി മാറ്റിയിട്ടുണ്ടെന്ന് ഉറപ്പ് വേണമെങ്കിൽ ശ്രദ്ധിക്കുക.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## പടി 3: ജനറേഷൻ പാരാമീറ്ററുകൾ ക്രമീകരിക്കുക

മോഡലിന്റെ ഔട്ട്പുട്ട് പെരുമാറ്റം നിയന്ത്രിക്കാൻ ജനറേഷൻ പാരാമീറ്ററുകൾ സജ്ജീകരിക്കുക. ഈ ക്രമീകരണങ്ങൾ ഫംഗ്ഷൻ കോൾ ചെയ്യുന്നതിനുള്ള നിശ്ചിതവും കേന്ദ്രീകൃതവുമായ പ്രതികരണങ്ങൾ ഉറപ്പാക്കുന്നു.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## പടി 4: ലഭ്യമായ ഫംഗ്ഷനുകൾ നിർവചിക്കുക

ഇവിടെ ഞങ്ങളുടെ AI അസിസ്റ്റന്റ് വിളിക്കാവുന്ന ഫംഗ്ഷനുകൾ നാം നിർവചിക്കുന്നു. ഈ ഉദാഹരണത്തിൽ, യാത്രാസംബന്ധമായ രണ്ട് ഫംഗ്ഷനുകൾ ഉണ്ട്:
1. **booking_flight_tickets**: വിമാനത്താവളങ്ങൾ തമ്മിലുള്ള ഫ്ലൈറ്റുകൾ ബുക്ക് ചെയ്യാനായി
2. **booking_hotels**: ഹോട്ടൽ താമസം ബുക്ക് ചെയ്യുന്നതിനായി

ഫംഗ്ഷൻ നിർവചനങ്ങൾ OpenAI-യുടെ ഫംഗ്ഷൻ കോൾ ചെയ്യൽ സ്കീമാ ഫോർമാറ്റ് അനുസരിച്ചാണ്.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## ഘട്ടം 5: വ്യാകരണം ജനറേഷൻ സഹായക ഫംഗ്ഷനുകൾ

ഈ സഹായക ഫംഗ്ഷനുകൾ ഞങ്ങളുടെ ഫംഗ്ഷൻ നിർവചനങ്ങളെ Lark വ്യാകരണ ഫോർമാറ്റിലേക്ക് മാറ്റുന്നു, അത് ONNX Runtime GenAI-ൽ മാർഗ്ഗനിർദ്ദേശിച്ച ജനറേഷനിൽ ഉപയോഗിക്കുന്നു. ഇത് മോഡൽ ശരിയായ JSON ഫോർമാറ്റിൽ സാധുവായ ഫംഗ്ഷൻ കോളുകൾ ഔട്ട്പുട്ട് ചെയ്യുന്നതായി ഉറപ്പാക്കുന്നു.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## പടി 6: വ്യാകരണമുണ്ടാക്കൽ പരീക്ഷിക്കുക

നമുക്ക് നമ്മുടെ വ്യാകരണമുണ്ടാക്കൽ ഫംഗ്ഷനുകൾ പരീക്ഷിച്ച് അവ നമ്മുടെ ടൂൾ നിജീകരണങ്ങളെ ശരിയായ ഫോർമാറ്റിലേക്ക് എങ്ങനെ മാറ്റുന്നു എന്ന് നോക്കാം.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## പടി 7: സിസ്റ്റം പ്രോംപ്റ്റും ജനറേറ്ററും തയ്യാറാക്കുക

ഇപ്പോൾ നാം മോഡലിന് ലഭ്യമായ ടൂളുകൾക്കുറിച്ച് 알려ിക്കുന്ന സിസ്റ്റം പ്രോംപ്റ്റ് സൃഷ്ടിക്കുകയും, ഗൈഡഡ് ജനറേഷൻ പരാമീറ്ററുകൾ ഉപയോഗിച്ച് ജനറേറ്റർ സജ്ജമാക്കുകയും ചെയ്യാം.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## പടി 8: മാർഗ്ഗനിർദ്ദേശിത ജനറേഷൻ ഉപയോഗിച്ച് ജനറേറ്റർ പ്രാരംഭിക്കുക

ഇപ്പോൾ നാം ക്രമീകരിച്ച പരാമീറ്ററുകൾ ഉപയോഗിച്ച് ജനറേറ്റർ സൃഷ്ടിച്ച് മാർഗ്ഗനിർദ്ദേശിത ജനറേഷനായി Lark വ്യാകരണം പ്രയോഗിക്കാം.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## ഘട്ടം 9: സമാന്തര ഫംഗ്ഷനുകൾ വിളിക്കൽ പരീക്ഷിക്കുക

ഇപ്പോൾ നിരവധി ഫംഗ്ഷനുകൾ വിളിക്കേണ്ടതുള്ള ഒരു സങ്കീർണ്ണമായ ട്രാവൽ ബുക്കിംഗ് സാഹചര്യത്തോടെ നമ്മുടെ ക്രമീകരണം പരീക്ഷിക്കാം.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### ഫോർമാറ്റുചെയ്‌ത ഉപയോക്തൃ പ്രോംപ്റ്റ് കാണുക


### ഫംഗ്ഷൻ കോളുകൾ സൃഷ്ടിക്കുക

മോഡൽ ഇപ്പോൾ നമ്മുടെ ഉപയോക്തൃ അഭ്യർത്ഥനയുടെ അടിസ്ഥാനത്തിൽ ഘടനാപരമായ ഫംഗ്ഷൻ കോളുകൾ സൃഷ്ടിക്കും. മാർഗദർശിത ജനറേഷനിന്റെ സഹായത്താൽ, ഔട്ട്‌പുട്ട് നേരിട്ട് 실행ിക്കാവുന്ന സാധുവായ JSON ഫോർമാറ്റിലാണ് ഉണ്ടാകുക.

**Expected Output**: മോഡൽ താഴെപ്പറയുന്ന ഫംഗ്ഷൻ കോളുകൾ സൃഷ്ടിക്കണം:
1. `booking_flight_tickets` Beijing (PEK) മുതൽ Paris (CDG) വരെയുള്ള വിശദാംശങ്ങളോടുകൂടി
2. `booking_hotels` പാരിസിലെ താമസത്തിന്റെ വിശദാംശങ്ങളോടുകൂടി

താഴെയുള്ള സെൽ പ്രവർത്തിപ്പിച്ച് ലൈവ് ജനറേഷൻ കാണുക:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## നിഗമനം

🎉 **അഭിനന്ദനങ്ങൾ!** ONNX Runtime GenAI ഉപയോഗിച്ച് Phi-4 Mini ഉപയോഗിച്ച് സമാന്തര ഫംഗ്ഷൻ കോളിംഗ് നിങ്ങൾ വിജയകരമായി നടപ്പിലാക്കിയിട്ടുണ്ട്.

### നിങ്ങൾ പഠിച്ച കാര്യങ്ങൾ:

1. **മോഡൽ ക്രമീകരണം**: Phi-4 Mini നെ ONNX Runtime GenAI ൽ എങ്ങനെ ക്രമീകരിക്കാമെന്ന്
2. **ഫംഗ്ഷൻ നിർവചനം**: AI ഫംഗ്ഷൻ കോളിംഗിന് ഫംഗ്ഷൻ സ്കീമുകൾ എങ്ങനെ നിർവചിക്കാമെന്ന്
3. **ഗൈഡഡ് ജനറേഷൻ**: സുസംരചിത ഔട്ട്പുട്ട് സൃഷ്ടിക്കാൻ Lark grammar എങ്ങനെ ഉപയോഗിക്കാമെന്ന്
4. **സമാന്തര ഫംഗ്ഷൻ കോളുകൾ**: നിരവധി ഫംഗ്ഷൻ കോളുകൾ ആവശ്യമായ സങ്കീർണ അഭ്യർത്ഥനകൾ എങ്ങനെ കൈകാര്യം ചെയ്യാമെന്ന്

### പ്രധാന ഗുണങ്ങൾ:

- ✅ **ഘടനാപരമായ ഔട്ട്പുട്ട്**: നിർദ്ദേശിത ജനറേഷൻ സാധുവായ JSON ഫംഗ്ഷൻ കോളുകൾ ഉറപ്പാക്കുന്നു
- ✅ **സമാന്തര പ്രോസസ്സിംഗ്**: ഒരേ അഭ്യർത്ഥനയിൽ متعدد ഫംഗ്ഷൻ കോളുകൾ കൈകാര്യം ചെയ്യുക
- ✅ **ഉയർന്ന പ്രകടനം**: ONNX Runtime നു ഒപ്റ്റിമൈസ്ഡ് ഇൻഫറൻസ് ലഭ്യമാക്കുന്നു
- ✅ **ലവച്യമായ സ്കീമ**: ഫംഗ്ഷൻ നിർവചനങ്ങൾ ചേർക്കാനും മാറ്റാനും എളുപ്പമാണ്

### സ്രോതസ്സുകൾ:

- [Phi-4 Mini ഡോക്യുമെന്റേഷൻ](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI ഡോക്യുമെന്റേഷൻ](https://onnxruntime.ai/docs/genai/)
- [ഫംഗ്ഷൻ കോളിംഗിന്റെ മികച്ച രീതികൾ](https://platform.openai.com/docs/guides/function-calling)


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
ഡിസ്‌ക്ലെയിമർ:
ഈ ഡോക്യുമെന്റ് [Co-op Translator](https://github.com/Azure/co-op-translator) എന്ന എ.ഐ. വിവർത്തന സേവനം ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് പരിശ്രമിച്ചിട്ടുണ്ടെങ്കിലും, യന്ത്രവൽക്കൃത വിവർത്തനങ്ങളിൽ പിശകുകളും അപകൃതതകളും ഉണ്ടായിരിക്കാവുന്നതാണ്. മൂലഭാഷയിലെ യഥാർത്ഥ ഡോക്യുമെന്റിനെ അധികാരപരമായ ഉറവിടമായി കരുതേണ്ടതാണ്. നിർണ്ണായകമായ വിവരങ്ങൾക്ക് പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനമാണ് ശുപാർശ ചെയ്യപ്പെടുന്നത്. ഈ വിവർത്തനത്തിന്റെ ഉപയോഗത്തിൽ നിന്നുണ്ടാകുന്ന ഏതൊരു തെറ്റിദ്ധാരണത്തിന്റേയും തെറ്റായ വ്യാഖ്യാനത്തിന്റേയും ഉത്തരവാദിത്വം ഞങ്ങളോട് ഉണ്ടാവില്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
